# Netflix Movies analysis project

## Target of project
- Practice using libraries like pandas, matplotlib
- Practice explaining targeted problems
- Practice visualization

## Data
Data I used comes from a website called "Kaggle" which is a data science community with resources
The file i used can be found at link: https://www.kaggle.com/datasets/rahulvyasm/netflix-movies-and-tv-shows

## Data description:
- show_id: A unique identifier for each title.
- type: The category of the title, which is either 'Movie' or 'TV Show'.
- title: The name of the movie or TV show.
- director: The director(s) of the movie or TV show. (Contains null values for some entries, especially TV shows where this information might not be applicable.)
cast: The list of main actors/actresses in the title. (Some entries might not have this information.)
- country: The country or countries where the movie or TV show was produced.
- date_added: The date the title was added to Netflix.
- release_year: The year the movie or TV show was originally released.
- rating: The age rating of the title.
- duration: The duration of the title, in minutes for movies and seasons for TV shows.
- listed_in: The genres the title falls under.
- description: A brief summary of the title.
